**Run the codes in this sequence: A, B, C, D, E, F, G, H, I, J, K.**

# (A) Import libraries

In [1]:
# os.chdir(r'C:/Users/UWAdmin/Desktop/TEST2/line codes')

from pymeasure.instruments.lighthousephotonics import Sprout #import laser codes
laser_power = Sprout('COM4')
laser_power.adapter.connection.baud_rate = 19200
laser_power.adapter.connection.read_termination = '\r'
laser_power.adapter.connection.write_termination = '\r'
laser_power.power = 0.01


C:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


In [14]:
import time
from datetime import timedelta
import rpy2.robjects as robjects

from newportxps import NewportXPS #motion controller
from threading import Thread
# from thread import MotionThread, MirrorThread
import os

import numpy as np
import pandas as pd
from write_utils import write_data_files, get_mean, get_move_y, duplicate_to_dataset

from flipper import mirror #import flipper mirror codes
from spectra import capture_photo #import spectrometer codes

from pathlib import Path


In [3]:
path = Path(r'c:\\Users\\UWAdmin\\Desktop\\_pyControl\\campaigns')
campaign_path = path / "2022-04-19-Series-1"
os.chdir(campaign_path)

# (B) Activate motion controller

In [4]:
xps = NewportXPS('192.168.254.254')
print(xps.status_report())

#pos_all() to get position of all axes
def pos_all():
    for sname, _ in xps.stages.items():
        print('{}: {}'.format(sname, xps.get_stage_position(sname)))
        

xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()

# XPS host:         192.168.254.254 (XPS-1B81)
# Firmware:         XPS-D-N13019
# Current Time:     Wed Apr 20 07:14:42 2022
# Last Reboot:      Thu Mar  3 03:58:16 2022
# Trajectory Group: XYZ
# Groups and Stages
XYZ (multipleaxesinuse), Status: Ready state from motion
   XYZ.X (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK
   XYZ.Y (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK
   XYZ.Z (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: First driver powered on - ZM low level
      Positioner Errors: OK


In [21]:
# from pymeasure.instruments.edwards import NXDS
# pump = NXDS('ASRL5::INSTR')
# pump.stop()

C:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


In [5]:
# xps.kill_group('XYZ')
# xps.initialize_allgroups()
# xps.home_allgroups()
xps.move_stage('XYZ.X', 2.5)
xps.move_stage('XYZ.Y', 6.5)
xps.move_stage('XYZ.Z', 2.1)
pos_all()

XYZ.X: 2.5000048512
XYZ.Y: 6.500016153
XYZ.Z: 2.0997860184


In [6]:
mirror("off")

MIRROR DOWN


## (C) Switch on the spectrometer

In [7]:
#Before running this code check that AddInProcess.exe, LightFieldAssistant, UntitledLightFieldExperiment is running (closed if open)
#Close anything related to Light Field

capture_photo("start",2,1,0) # the values 2,1,0 don't matter, they are later used for indexing tasks

## (D) Set chamber pressure
#Used to input new Sample
#Move this before auto focusing

In [8]:
from pressure import close_valve,open_valve,close_all,current_pressure,gopr, to_ambient, to_vacuum, quick_fill
current_pressure()

C:\Users\UWAdmin\Anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


201

In [12]:
to_ambient()

9

In [15]:
t = 2 # in s
to_ambient()
time.sleep(1)
to_ambient()
to_vacuum(t)
quick_fill()
time.sleep(1)
to_vacuum(t)


2

# (E) To make sure that the material gets laser beam of max. intensity , focusing of z axis needs to be properly adjusted. With LightField running, use the following code to find z, enter 0 and confirm with yes to end.

In [9]:
mirror('on')

#Iterate for input to find z value
print("Enter z value (enter 0 when done):")
z=''
while z != 0:
    if z == '':
        z = float(input(':'))
    print(z)
    xps.move_stage('XYZ.Z', z)
    xps.move_stage('XYZ.X', 5)
    xps.move_stage('XYZ.Y', 7)
    z = float(input(':'))
    
# mirror('on')
pos_all()
while input('Is LightField Scan Stopped?:(yes/no)')!='yes':
    continue

MIRROR UP
Enter z value (enter 0 when done):
1.45
1.2
1.6
1.9
2.0
2.5
2.2
2.1
2.05
2.18
2.15
2.13
2.12
2.11
2.105
2.11
2.105
XYZ.X: 4.999992003
XYZ.Y: 7.0000065036
XYZ.Z: 2.1050073414


# (F) Prepare initial dataset

## MAKE SURE THE BOUNDS ARE COMMENSURATE WITH R CODE PARAM SET

In [7]:
# from pathlib import Path
# path = Path(r'c:\\Users\\UWAdmin\\Desktop\\_pyControl\\campaigns')
# write_data_files(path, nr_random_lines=1, series=2)

# (G) Define coordinate details

**Takes input for**  
  
-number of lines

-starting position of x

-starting position of y

-vertical gap/step size

<!-- # (G) Define coordinate details

**Takes input for**  
**COLUMN 1**  
-number of lines

-starting position of x

-starting position of y

-vertical gap/step size

-**COLUMN 2** - yes/no

-starting position of x

-starting position of y

-vertical gap/step size -->

In [9]:
lines, start_y, step_y = 7, 6.5, 1
move_y = get_move_y(lines, start_y)
print("Lines will be made at following y axis:", move_y,"\n")

Lines will be made at following y axis: [6.5, 7.5, 8.5, 9.5, 10.5, 11.5, 12.5] 



## THREADING

In [16]:
#Change this to motion_thread
class MotionThread(Thread):
   
    def run(self):
        import pandas as pd
        d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        total_time=np.sum(d['ramptime'])
        
        if d['ramptime'][0]<0.5:
            time.sleep(.5-d['ramptime'][0])
        print("MOTION THREAD\n")
        print("XPS run trajectory")
        xps.run_trajectory('foreward',)
        print('Total time {} in class'.format(total_time))
        time.sleep(total_time)
        print("TOTAL_TIME DONE, LASER 0.05W")
        laser_power.power=0.01
        print("finished and current position is:\n")
        pos_all()

#Change this to mirror_thread
class MirrorThread(Thread):
    """MOTOR THREAD"""
    def run(self):
        import pandas as pd
        d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        mirror_sleep=d['ramptime'][0]-.5
        print("MIRROR THREAD")
        for i in range(2):
            if i==0:
                print(d['ramptime'][0])
                if d['ramptime'][0]<0.5:
                    print("mirror on")
       
                    start_time = time.monotonic()
                    mirror('on')
       
                    end_time = time.monotonic()
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line    

                else:
                    print('{} > 0.5'.format(d['ramptime'][0]))
                    print('Mirror sleep {} in class'.format(mirror_sleep))
                    time.sleep(mirror_sleep)
                
                    print("mirror on")
                    start_time = time.monotonic()
                    mirror('on')
                    end_time = time.monotonic()
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line
            else:
                mirror('off')
                print("mirror off")

# (K) Begin the experiment

In [12]:
#path=r'C:\Users\UWAdmin\Desktop\TEST2\Campaigns\2022-04-11-Sample1'
#os.chdir(path)

In [13]:
col = 1
TOTAL_LINES = len(move_y)
steps = 0
for move_line in range(TOTAL_LINES):
        # print("For this print this is line #: ",move_line)
        # print("Experiment Line number: " ,move_line)
        
        save_line = col*TOTAL_LINES + move_line
        steps = save_line * 9
        print (f"before: {move_line}, {save_line}, {steps}")
        
        # print (f"after : {move_line}, {save_line}, {steps}")

before: 0, 7, 63
before: 1, 8, 72
before: 2, 9, 81
before: 3, 10, 90
before: 4, 11, 99
before: 5, 12, 108
before: 6, 13, 117


In [22]:
def main_experiment(startx: float, move_y: list, col: int) -> None:
    """Starts the experiment. Currently works ok with equal length of col1 and col2 lines.
       startx: Starting position on the x-axis
       starty: Starting position on the y-axis
       step_y: The distance between lines on the y-axis
       move_y: List of positions on the y-axis
       col_no: Defines the column number"""

    # line_name_start = col_no*len(move_y) #0,1,   # What the first line # will be named
    # line_name_end = line_name_start + len(move_y) #What the last line # will be named
    
    import pandas as pd
    print(f'LOADING DATA.CSV FOR COLUMN: {col}') 
    df2 = pd.read_csv('data.csv')   # to avoid pd referenced before assignment
    print (df2)
    
    print("EXPERIMENT IS STARTING\n")
    import time
    time.sleep(5)
    _MIRROR_MOVE_TIME = .5
    _LINE_LENGTH = 2
    VACUUM_TIME = 0.5 # in minutes

    #set this to zero!!!!!!!
    #starting_line = 4
    STARTING_LINE = 0
    _SPOTS_MEASURED = 9
    TOTAL_LINES = len(move_y) #+STARTING_LINE
    RANDOM_LINES = 7
    TARGET = 'GD'

    print(f"Here's the number of total lines:{TOTAL_LINES}")
    
    for move_line in range(STARTING_LINE,TOTAL_LINES):
        print(f"====== MOVE LINE: {move_line} ======")
        
        save_line = col * TOTAL_LINES + move_line
        steps = save_line * _SPOTS_MEASURED
        print(f"====== SAVE LINE: {save_line} ======")

        # move the axes to their start position
        xps.move_stage('XYZ.Y',move_y[move_line])
        pos_all()
        print("\n\n")
        df2 = pd.read_csv('data.csv')
        plot_data_df = pd.read_csv('plot_data.csv')
        print(f"\n data.csv file before index {save_line}:\n")
        print(df2)
        print(f"\n plot_data.csv file before index {save_line}:\n")
        print(plot_data_df,'\n')
        # line = df2['ratio'].count() #find in data.csv what line number we're on
        
        # set time
        time_of_file = df2['time'][save_line]
        print(f"\nCurrent patterning time is: {time_of_file} ms")

        # vacuum-infill
        print("Vacuum-infill in progress...")
        # t = 1 # in minutes
        to_ambient()
        print(f"1st ambient: {current_pressure()}")
        time.sleep(1)
        to_ambient()
        print(f"2nd ambient: {current_pressure()}")
        time.sleep(1)
        to_ambient()
        print(f"3rd ambient: {current_pressure()}")
        time.sleep(5)
        to_vacuum(VACUUM_TIME)
        print(f"1st vac: {current_pressure()}")
        time.sleep(5)
        quick_fill()
        print(f"After quick_fill: {current_pressure()}")
        time.sleep(1)
        to_ambient()
        print(f"3rd ambient: {current_pressure()}")
        time.sleep(2)
        to_vacuum(VACUUM_TIME)
        time.sleep(5)
        print(f"2nd vac : {current_pressure()}")
        
        # set pressure
        pressure_of_file=df2['pressure'][save_line]
        for j in range(3):
            gopr(pressure_of_file)
            close_all()
            time.sleep(5)
        print(f"\n Pressure is now :{pressure_of_file}\n")

        xps.define_line_trajectories(start=float(startx),
                                 stop=float(startx)+_LINE_LENGTH,
                                 step=0.01,
                                 scantime=(time_of_file/1000),
                                 axis='X')
        # downloads trajectory defined above
        xps.download_trajectory('foreward.trj')
        import pandas as pd
        d = pd.read_csv('foreward.trj', header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        print(d)

        total_time = np.sum(d['ramptime'])
        mirror_sleep = d['ramptime'][0] - _MIRROR_MOVE_TIME
        print(f'Total time: {total_time}, Mirror_sleep: {mirror_sleep} in MAIN')

        # power will be set to assigned value, make sure mirror is OFF (!) \
        # and power is given enough time to reach it's value

        power_of_file = df2["power"][save_line]


    ###########################

        # print("#############STARTING PRE-MEASUREMENT ###########\n")
        # check=[float(startx) + .2*_LINE_LENGTH, \
        #         float(startx) + .4*_LINE_LENGTH, \
        #         float(startx) + .6*_LINE_LENGTH]
        # # mm=0
        # for spot in range(3):
        #     print(f"########## spot: {spot} ##############")
        #     xps.move_stage('XYZ.X',check[spot])
        #     print(pos_all())

        #     for iii in range(3):
        #         print(f"############## iteration: {iii} ################")
        #         mirror("on")
        #         #writing G/D of 3 spots in 3 lines inside dataset.csv
        # #         capture_photo("on",spot,line)
        #         GD=capture_photo("first",spot,line,iii, target=TARGET)

        #         mirror("off")
        #         time.sleep(5)

    ############################

    # PATTERN EXPERIMENT


        print("Power is now: {} mW".format(power_of_file))
        xps.move_stage('XYZ.X',(float(startx) - d['rampdist'][0]))
        pos_all()
        time.sleep(15)
        print("\n\n")
        mirror("off")
        a=power_of_file
        #laser_power.power=((a-5.7488)/0.138859)/1000
        laser_power.power = (a/0.0994)/1000
        time.sleep(15)
        
        
        print("###### Starting pattern in 5 seconds! #########")
        ##Draw Lines
        time.sleep(10)
        
        import sys
        motion_thread=MotionThread() # MOTION THREAD
        mirror_thread=MirrorThread() # MIRROR THREAD
        # stop_threads = False
        motion_thread.start()
        mirror_thread.start()

        motion_thread.join()
        mirror_thread.join()

        time.sleep(10)
        print("\n Job done\n")

        ##raman spectra analysis
        laser_power.power=0.01
        time.sleep(15)

    ########################################

    # post patterning raman analysis

        check=[float(startx) + .2*_LINE_LENGTH, \
                float(startx) + .4*_LINE_LENGTH, \
                float(startx) + .6*_LINE_LENGTH]
        # mm=0
        for spot in range(3):
            print(f"############### SPOT {spot} ##################")
            xps.move_stage('XYZ.X',check[spot])
            print(pos_all())

            for iii in range(3):
                print(f"############# iteration {iii} ###############")
                mirror("on")
                #writing G/D of 3 spots in 3 lines inside dataset.csv
        #         capture_photo("on",spot,line)
                print(f"Measuring patterned line for {TARGET}")
                GD = capture_photo("on", spot, save_line, iii, target=TARGET)

                mirror("off")
                time.sleep(5)      

        if save_line <= RANDOM_LINES-1:
            print(f"############# LINE {save_line} <= 3, CURRENT_STEP={steps}, TAKING MEAN: #############")
           take_mean(steps, save_line)

        if save_line > RANDOM_LINES-1:
            print(f"################ LINE {save_line} > 3, UPDATING MBO ##########################")
            rsum = robjects.r['experiment'] # added because UnboundLocalError: local variable 'rsum' referenced before assignment
            mean_ratio =take_mean(steps, save_line) # writes in data.csv mean from 9 rows in dataset.csv
            print(f"%%%%%%%%%% MEAN_RATIO: {mean_ratio}, STEPS: {steps} %%%%%%%%%%%%%")
            # print(mean_ratio, steps)
            if (mean_ratio is None) | (np.isnan(mean_ratio)):
                rsum((0))
            else:
                mean_ratio = float(mean_ratio)
                rsum((mean_ratio))

            duplicate_to_dataset(nr_proposed_lines=7)

        if save_line >= RANDOM_LINES-1:
            print("#####################\n")
            print(f"LINE {save_line}: DEFINING PROPOSED LINE IN DATA AND PLOT_DATA")
            robjects.r('''
                    experiment<-function(s){
 
                    presToR <- function (x) {(x - 50) / 10}
                    presToExp <- function (x) {x * 10 + 50}

                    if(s==1){ #BO called first time to initialize and propose first point
                    require(XML)

                      suppressWarnings({suppressMessages({
                        library(mlrMBO)
                        library(ggplot2)
                        library(doMC)
                      })})

                      ps = makeParamSet(
                          makeIntegerParam("power", lower = 1, upper = 1190),
                          makeIntegerParam("time", lower = 1050, upper = 7500),
                          makeIntegerParam("pressure", lower = 1, upper = 30, trafo = function(x) x * 10 + 50)
                        )

                      ctrl = makeMBOControl(y.name = "ratio")
                      ctrl = setMBOControlInfill(ctrl, opt = "focussearch", opt.focussearch.maxit = 10, opt.focussearch.points = 10000, crit = makeMBOInfillCritEI())

                      # dataset-2 should be replaced and later this line as well
                      data=read.csv("data.csv")
                    
                      data$pressure <- presToR(data$pressure)

                      # data<-na.omit(data)
                   
                      suppressMessages({opt.state = initSMBO(par.set = ps,design = data, control = ctrl, minimize = FALSE, noisy = TRUE)})

                      print("Proposed parameters:")
                      prop <- suppressWarnings({proposePoints(opt.state)})
                      print(prop$prop.points) 
                      print(prop$crit.components)
                      saveRDS(prop,'./prop.rds')
                      saveRDS(opt.state,'./opt.state.rds')
                      dummy_ratio = 0
                      cat("Expected value (upper bound):\n")
                      cat(paste(prop$crit.components$mean, " (", \
                      prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))
                    
                      prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)

                      x<-c(prop$prop.points, dummy_ratio, prop$crit.components$mean, prop$crit.components$mean + prop$crit.components$se, \
                          prop$crit.components$mean + prop$crit.components$mean + prop$crit.components$se)

                      x2<-c(prop$prop.points)
                      #data_plot=read.csv("plot_data.csv")
                      
                      

                      #dataset-2 should be replaced and later this line as well
                      write.table(x2, file = "data.csv", sep = ",",\
                        append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                      write.table(x, file = "plot_data.csv", sep = ",",\
                        append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                    }
                    else{
                    
                      prop = readRDS('./prop.rds')
                      opt.state = readRDS('./opt.state.rds')
                      library(mlrMBO)
                      print(prop$crit.components)
                      print(prop$prop.points)
                      print("Proposed parameters:")
                      updateSMBO(opt.state, x = prop$prop.points, y = s)
                      prop <- suppressWarnings({proposePoints(opt.state)})
                      print(prop$prop.points) 
                      print(prop$crit.components)
                      saveRDS(prop,'./prop.rds')
                      saveRDS(opt.state,'./opt.state.rds')
                      dummy_ratio=0
                      cat("Expected value (upper bound):\n")
                      cat(paste(prop$crit.components$mean, " (", prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))
                        
                      prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)  
                    
                      x<-c(prop$prop.points, dummy_ratio, prop$crit.components$mean, prop$crit.components$mean + prop$crit.components$se, \
                          prop$crit.components$mean + prop$crit.components$mean + prop$crit.components$se)
                      x2<-c(prop$prop.points)
                      #data_plot=read.csv("plot_data.csv")

                      #dataset-2 should be replaced and later this line as well
                      write.table(x2, file = "data.csv", sep = ",",\
                           append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)
                      write.table(x, file = "plot_data.csv", sep = ",",\
                           append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                    }}

                    ''')
            print(f"......LINE {save_line} ENDS.....")

            
        if save_line == RANDOM_LINES-1: ##AI wil start working when initial design has 4 data (0,1,2,3= 4 data); change if init data > 4
            print(f"############ LINE {save_line} == 3, FIRST MBO ###############")
            rsum = robjects.r['experiment']
            rsum((1))
            duplicate_to_dataset(nr_proposed_lines=7)

In [17]:
robjects.r('''
                    experiment<-function(s){
 
                    presToR <- function (x) {(x - 50) / 10}
                    presToExp <- function (x) {x * 10 + 50}

                    if(s==1){ #BO called first time to initialize and propose first point
                    require(XML)

                      suppressWarnings({suppressMessages({
                        library(mlrMBO)
                        library(ggplot2)
                        library(doMC)
                      })})

                      ps = makeParamSet(
                          makeIntegerParam("power", lower = 1, upper = 1190),
                          makeIntegerParam("time", lower = 1050, upper = 7500),
                          makeIntegerParam("pressure", lower = 1, upper = 30, trafo = function(x) x * 10 + 50)
                        )

                      ctrl = makeMBOControl(y.name = "ratio")
                      ctrl = setMBOControlInfill(ctrl, opt = "focussearch", opt.focussearch.maxit = 10, opt.focussearch.points = 10000, crit = makeMBOInfillCritEI())

                      # dataset-2 should be replaced and later this line as well
                      data=read.csv("data.csv")
                    
                      data$pressure <- presToR(data$pressure)

                      # data<-na.omit(data)
                   
                      suppressMessages({opt.state = initSMBO(par.set = ps,design = data, control = ctrl, minimize = FALSE, noisy = TRUE)})

                      print("Proposed parameters:")
                      prop <- suppressWarnings({proposePoints(opt.state)})
                      print(prop$prop.points) 
                      print(prop$crit.components)
                      saveRDS(prop,'./prop.rds')
                      saveRDS(opt.state,'./opt.state.rds')
                      dummy_ratio = 0
                      cat("Expected value (upper bound):\n")
                      cat(paste(prop$crit.components$mean, " (", \
                      prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))
                    
                      prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)

                      x<-c(prop$prop.points, dummy_ratio, prop$crit.components$mean, prop$crit.components$mean + prop$crit.components$se, \
                          prop$crit.components$mean + prop$crit.components$mean + prop$crit.components$se)

                      x2<-c(prop$prop.points)
                      #data_plot=read.csv("plot_data.csv")
                      
                      

                      #dataset-2 should be replaced and later this line as well
                      write.table(x2, file = "data.csv", sep = ",",\
                        append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                      write.table(x, file = "plot_data.csv", sep = ",",\
                        append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                    }
                    else{
                    
                      prop = readRDS('./prop.rds')
                      opt.state = readRDS('./opt.state.rds')
                      library(mlrMBO)
                      print(prop$crit.components)
                      print(prop$prop.points)
                      print("Proposed parameters:")
                      updateSMBO(opt.state, x = prop$prop.points, y = s)
                      prop <- suppressWarnings({proposePoints(opt.state)})
                      print(prop$prop.points) 
                      print(prop$crit.components)
                      saveRDS(prop,'./prop.rds')
                      saveRDS(opt.state,'./opt.state.rds')
                      dummy_ratio=0
                      cat("Expected value (upper bound):\n")
                      cat(paste(prop$crit.components$mean, " (", prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))
                        
                      prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)  
                    
                      x<-c(prop$prop.points, dummy_ratio, prop$crit.components$mean, prop$crit.components$mean + prop$crit.components$se, \
                          prop$crit.components$mean + prop$crit.components$mean + prop$crit.components$se)
                      x2<-c(prop$prop.points)
                      #data_plot=read.csv("plot_data.csv")

                      #dataset-2 should be replaced and later this line as well
                      write.table(x2, file = "data.csv", sep = ",",\
                           append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)
                      write.table(x, file = "plot_data.csv", sep = ",",\
                           append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

                    }}

                    ''')

<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x000001F662637388> [RTYPES.CLOSXP]
R classes: ('function',)

In [18]:
rsum = robjects.r['experiment']

In [23]:
import sys

campaign_path = path / "2022-04-19-Series-1"
os.chdir(campaign_path)


first_column = int(input("First column number:"))
first_column_verify = int(input("Verify first column number:"))
while first_column !=first_column_verify:
    first_column = int(input("Error, numbers don't match for first column. Re-enter first column number:"))
    first_column_verify = int(input("Verify first column number:"))

second_column = int(input("Second column number:"))
second_column_verify = int(input("Verify second column number:"))

while second_column !=second_column_verify:
    second_column = int(input("Error, numbers don't match for second column. Re-enter second column number:"))
    second_column_verify = int(input("Verify second column number:"))


intensity_check =capture_photo('adjust',0,0,0)
print("---Final Verification of Experiment---")
print("Current Pressure: ", current_pressure())
pos_all()
print("Checked Intesity: ",intensity_check)
print(f"Print to column: {first_column} and column {second_column}")
print(os.getcwd())



verify_start = input("Do you approve pressure, intensity, and column numbers (yes/no)")
if verify_start == "yes":
    "Pressure, intensity, and column numbers have been approved. Starting experiment..."
    startx1 = 2.5
    main_experiment(startx1, move_y, col=first_column)

    # startx2 = 5.5
    # main_experiment(startx2, move_y, col=second_column)
else:
    sys.exit()


MIRROR UP
 
MIRROR DOWN
---Final Verification of Experiment---
Current Pressure:  2
XYZ.X: 2.5000048512
XYZ.Y: 6.4999984536
XYZ.Z: 2.0999984112
Checked Intesity:  215.66666666666666
Print to column: 1 and column 2
c:\Users\UWAdmin\Desktop\_pyControl\campaigns\2022-04-19-Series-1
LOADING DATA.CSV FOR COLUMN: 1
   power  time  pressure     ratio
0    143  3217       170  0.819253
1    159  1958       100  0.857771
2    238  3549       220  1.333680
3   1079  2802       230  1.242668
4    724  2631       210  1.190802
5    839  4650       220  0.754259
6    278  3269       220  0.805314
7      1  3594       150       NaN
EXPERIMENT IS STARTING

Here's the number of total lines:7
====== MOVE LINE: 0 ======
====== SAVE LINE: 7 ======
XYZ.X: 2.5000048512
XYZ.Y: 6.4999984536
XYZ.Z: 2.0999984112




 data.csv file before index 7:

   power  time  pressure     ratio
0    143  3217       170  0.819253
1    159  1958       100  0.857771
2    238  3549       220  1.333680
3   1079  2802       230 

R[write to console]: Loading required package: mlr

R[write to console]: Loading required package: ParamHelpers

R[write to console]: Warning message: 'mlr' is in 'maintenance-only' mode since July 2019.
Future development will only happen in 'mlr3'
(<https://mlr3.mlr-org.com>). Due to the focus on 'mlr3' there might be
uncaught bugs meanwhile in {mlr} - please consider switching.

R[write to console]: Loading required package: smoof

R[write to console]: Loading required package: checkmate



         se     mean
1 0.2164809 1.017863
     power time pressure
4679   679 2096       30
[1] "Proposed parameters:"
    power time pressure
117   909 3553       18
          se     mean
1 0.06377776 1.363863
Expected value (upper bound):
1.36386253958464 (1.42764029834853)
#####################

LINE 7: DEFINING PROPOSED LINE IN DATA AND PLOT_DATA
......LINE 7 ENDS.....
====== MOVE LINE: 1 ======
====== SAVE LINE: 8 ======
XYZ.X: 3.7000064718
XYZ.Y: 7.4998198602
XYZ.Z: 2.0999984112




 data.csv file before index 8:

   power  time  pressure     ratio
0    143  3217       170  0.819253
1    159  1958       100  0.857771
2    238  3549       220  1.333680
3   1079  2802       230  1.242668
4    724  2631       210  1.190802
5    839  4650       220  0.754259
6    278  3269       220  0.805314
7      1  3594       150  1.072549
8    909  3553       230       NaN

 plot_data.csv file before index 8:

   power  time  pressure  dummy_ratio  pred_mean  pred_upper        ei
0      1  3594 

KeyboardInterrupt: 

In [21]:
col = 1
TOTAL_LINES = len(move_y)
steps = 0
for move_line in range(TOTAL_LINES):
        # print("For this print this is line #: ",move_line)
        # print("Experiment Line number: " ,move_line)
        
        save_line = col*TOTAL_LINES + move_line
        steps = save_line * 9
        print (f"before: {move_line}, {save_line}, {steps}")
        
        # print (f"after : {move_line}, {save_line}, {steps}")

before: 0, 7, 63
before: 1, 8, 72
before: 2, 9, 81
before: 3, 10, 90
before: 4, 11, 99
before: 5, 12, 108
before: 6, 13, 117


In [23]:
save_line, steps =  8, 63
SPOTS_MEASURED = 9
get_mean(steps,save_line,SPOTS_MEASURED)

c:\Users\UWAdmin\Desktop\_pyControl\campaigns\2022-04-19-Series-1
Filling nans with 0
saving mean 1.5 to data.csv


1.5